In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
import datetime
import os
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import train_test_split

In [24]:
#https://stackoverflow.com/questions/41032551/how-to-compute-receiving-operating-characteristic-roc-and-auc-in-keras
def auroc(y_true, y_pred):
    return tf.py_func(roc_auc_score, (y_true, y_pred), tf.double)

In [25]:
#input variables
path = 'image_arrays'

#model variables
test_size = 0.25
batch_size = 32
epoch_number = 50
validation_split = 0.15
learning_rate = 1e-3

#architecture variables
"""
CNL1_filters = 32
CNL1_kernal_size = 5
MPL1_pool_size= (2,2)
MPL1_strides = 2
CNL2_filters = 64
CNL2_kernal_size = 5
MPL2_pool_size = (2,2)
MPL2_strides = 2
"""
#defined within because were modeling after a paper

'\nCNL1_filters = 32\nCNL1_kernal_size = 5\nMPL1_pool_size= (2,2)\nMPL1_strides = 2\nCNL2_filters = 64\nCNL2_kernal_size = 5\nMPL2_pool_size = (2,2)\nMPL2_strides = 2\n'

In [26]:
galaxyzoo = pd.read_csv("zoo2MainSpecz.csv", usecols=[8], nrows=10000)
Class = galaxyzoo["gz2class"].values
dictionary = {'A':int(2),'E':int(1),'S':int(0)}
#resave using my dictionary
target = np.empty((len(Class)))
for i in range(len(Class)):
    target[i] = int(dictionary[Class[i][0]])

In [27]:
#https://keras.io/preprocessing/image/

#datagen flow from directory wont work, need to create a different iterator. Have to combine with the targets somehow. 

#seems like a tf.data.Dataset is what we want... well, in the future atleast. lets right now get this working by doing something
#memory intensive.
#def generator(path,batchsize):
#    """
#    I want this function when called provide a 
#    """
#    image_list = os.listdir(path)
    

In [28]:
directory = os.listdir(path)
data = np.zeros((len(directory),64,64,5))
inbetween_target = np.zeros((len(directory)))
for i in range(len(directory)):
    ith = directory[i].split('_')[-1]
    ith = ith.split('.')[0] #we want the str of what element this is so I can grab from my target array the correct label
    ith = int(ith)
    array = np.load('{}\\{}'.format(path,directory[i]))
    data[i,:,:,:] = array
    inbetween_target[i] = target[ith]
target = inbetween_target
print(len(target))
print(np.shape(data))
print(len(directory))

7848
(7848, 64, 64, 5)
7848


In [29]:
train_images, test_images, train_target, test_target = train_test_split(data, target, test_size=test_size)

In [ ]:
#https://arxiv.org/pdf/1903.05580.pdf hyperspectral online data augmentation paper

In [30]:
#so this is pretty neat, you can create a keras callback to display on tensorboard using a simplified summary tf api

#and also this is an example of how to change the lr on the fly, which is pretty handy
#https://keras.io/callbacks/

logdir="summaries/scalars/" + str(datetime.datetime.now().timestamp())
tensorboard_callback = keras.callbacks.TensorBoard(log_dir=logdir,
                                                   histogram_freq=1,
                                                   write_graph=False,
                                                   write_grads=True,)
                                                   #write_images=True)
"""
file_writer = tf.summary.create_file_writer(logdir + "/metrics")
file_writer.set_as_default()

def lr_schedule(epoch):
"""
#Returns a custom learning rate that decreases as epochs progress.
"""
    learning_rate = 0.2
    if epoch > 10:
        learning_rate = 0.02
    if epoch > 20:
        learning_rate = 0.01
    if epoch > 50:
        learning_rate = 0.005

    tf.summary.scalar('learning rate', data=learning_rate, step=epoch)
    return learning_rate

lr_callback = keras.callbacks.LearningRateScheduler(lr_schedule)
tensorboard_callback = keras.callbacks.TensorBoard(log_dir=logdir)
"""


"\n    learning_rate = 0.2\n    if epoch > 10:\n        learning_rate = 0.02\n    if epoch > 20:\n        learning_rate = 0.01\n    if epoch > 50:\n        learning_rate = 0.005\n\n    tf.summary.scalar('learning rate', data=learning_rate, step=epoch)\n    return learning_rate\n\nlr_callback = keras.callbacks.LearningRateScheduler(lr_schedule)\ntensorboard_callback = keras.callbacks.TensorBoard(log_dir=logdir)\n"

In [31]:
def create_model(dropout_rate=dropout_rate, learning_rate=learning_rate):
    
    model = keras.Sequential([])
    
    model.add(keras.layers.Conv2D(input_shape=(64,64,5),filters=32,kernel_size=6,padding='same',activation=tf.nn.relu))
    model.add(keras.layers.Dropout(rate=0.5))
    
    model.add(keras.layers.Conv2D(filters=64,kernel_size=5,padding='same',activation=tf.nn.relu))
    model.add(keras.layers.MaxPool2D(pool_size=2,))
    model.add(keras.layers.Dropout(rate=0.25))
    
    model.add(keras.layers.Conv2D(filters=128,kernel_size=2,padding='same',activation=tf.nn.relu))
    model.add(keras.layers.MaxPool2D(pool_size=2,))
    model.add(keras.layers.Dropout(rate=0.25))
    
    model.add(keras.layers.Conv2D(filters=128,kernel_size=3,padding='same',activation=tf.nn.relu))
    model.add(keras.layers.Dropout(rate=0.25))

    model.add(keras.layers.Flatten())
    model.add(keras.layers.Dense(units=1024,activation=tf.nn.relu))
    model.add(keras.layers.Dense(units=64,activation=tf.nn.relu))
    model.add(keras.layers.Dense(units=3,activation=tf.nn.softmax))
    
    adam = keras.optimizers.Adam(lr=learning_rate)
    model.compile(optimizer=adam, loss='sparse_categorical_crossentropy',metrics=['accuracy',]) #auroc])
    return(model)

In [32]:
keras.backend.clear_session()
model = create_model(dropout_rate=dropout_rate, learning_rate = learning_rate)

In [33]:
#datagenerator will rotate flip and zoom my data for augmentations, b/c of cosmo principle I must be invarient to these.

#interestingly enough, there is no support for datagen right now with 5 channels:
#https://github.com/keras-team/keras/issues/4664
#that sucks, because if memory was an issue before, now I will have to default on doing my image roatations and zooms before
#hand, and that also takes up memory.

In [34]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 64, 64, 32)        5792      
_________________________________________________________________
dropout (Dropout)            (None, 64, 64, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 64, 64, 64)        51264     
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 32, 32, 64)        0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 32, 32, 64)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 32, 32, 128)       32896     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 16, 16, 128)       0         
__________

In [35]:
history = model.fit(train_images,
                    train_target,
                    batch_size=batch_size,
                    epochs=epoch_number,
                    validation_split=validation_split,
                    callbacks=[tensorboard_callback])#, lr_callback]) #and will display that on tensorboard too

Train on 5003 samples, validate on 883 samples
Epoch 1/50
5003/5003 [==============================] - 287s 57ms/sample - loss: 0.6831 - acc: 0.6196 - val_loss: 0.5368 - val_acc: 0.7542
Epoch 2/50
5003/5003 [==============================] - 285s 57ms/sample - loss: 0.5513 - acc: 0.7408 - val_loss: 0.4823 - val_acc: 0.7588
Epoch 3/50
5003/5003 [==============================] - 294s 59ms/sample - loss: 0.4814 - acc: 0.7883 - val_loss: 0.4507 - val_acc: 0.8086
Epoch 4/50
5003/5003 [==============================] - 287s 57ms/sample - loss: 0.4431 - acc: 0.8055 - val_loss: 0.4126 - val_acc: 0.8199
Epoch 5/50
5003/5003 [==============================] - 278s 56ms/sample - loss: 0.3980 - acc: 0.8337 - val_loss: 0.4404 - val_acc: 0.8199
Epoch 6/50
5003/5003 [==============================] - 279s 56ms/sample - loss: 0.3881 - acc: 0.8435 - val_loss: 0.4075 - val_acc: 0.8245
Epoch 7/50
5003/5003 [==============================] - 280s 56ms/sample - loss: 0.3662 - acc: 0.8419 - val_loss: 0.419

KeyboardInterrupt: 

In [ ]:
history.history.keys()

In [ ]:
# summarize history for accuracy
plt.plot(history.history['acc'])
plt.plot(history.history['val_acc'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.savefig('accuracy-2.png')
plt.show()
# summarize history for loss
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.savefig('loss-2.png')
plt.show()

In [ ]:
test_loss, test_acc = model.evaluate(test_images, test_target)
print('Test accuracy:', test_acc)
print('Test loss:', test_loss)

In [ ]:
"""
model_1

test_train_split = 0.75
batch_size = 64
epoch_number = 20
validation_split = 0.15
learning_rate = 1e-3
dropout_rate = 0.40

output: terrible overtraining, but promising that in 20 epochs I hit 92%: reducing learning rate by a lot
"""

""""
model_2

test_train_split = 0.75
batch_size = 64
epoch_number = 20
validation_split = 0.15
learning_rate = 1e-6
dropout_rate = 0.40


Test accuracy: 0.60142714
Test loss: 0.7398002415622046

Epoch 20/20
5003/5003 [==============================] - 59s 12ms/sample - loss: 0.6610 - acc: 0.6874 - val_loss: 0.7216 - val_acc: 0.6625

Perhaps I was a little overzealous with the LR reduction: I am going to prop it back up, increase dropout.
"""

"""
model_3

test_train_split = 0.75
batch_size = 64
epoch_number = 20
validation_split = 0.15
learning_rate = 1e-4
dropout_rate = 0.45

1962/1962 [==============================] - 7s 4ms/sample - loss: 0.9262 - acc: 0.7457
Test accuracy: 0.7456677
Test loss: 0.9262316330249643

Epoch 19/20
5003/5003 [==============================] - 58s 12ms/sample - loss: 0.1947 - acc: 0.9310 - val_loss: 0.8856 - val_acc: 0.7610
Epoch 20/20
5003/5003 [==============================] - 58s 12ms/sample - loss: 0.1838 - acc: 0.9414 - val_loss: 0.9662 - val_acc: 0.7542

too high of LR: trying to lower by 1e-1
"""

"""
model_4

#model variables
test_train_split = 0.75
batch_size = 64
epoch_number = 20
validation_split = 0.15
learning_rate = 1e-5
dropout_rate = 0.45

Epoch 19/20
5003/5003 [==============================] - 60s 12ms/sample - loss: 0.4295 - acc: 0.8183 - val_loss: 0.6812 - val_acc: 0.7407
Epoch 20/20
5003/5003 [==============================] - 60s 12ms/sample - loss: 0.4200 - acc: 0.8279 - val_loss: 0.6655 - val_acc: 0.7293

1962/1962 [==============================] - 7s 4ms/sample - loss: 0.6960 - acc: 0.6774
Test accuracy: 0.67737
Test loss: 0.6960039274286665

very weird that this has a lower accuracy, but also much lower loss than model_3, which clearly had too high a LR
:Increasing LR slightly, just to see where I can maximize it and still be stable.
"""

"""
model_5

test_train_split = 0.75
batch_size = 64
epoch_number = 20
validation_split = 0.15
learning_rate = 3e-5
dropout_rate = 0.45

Epoch 19/20
5003/5003 [==============================] - 60s 12ms/sample - loss: 0.3353 - acc: 0.8701 - val_loss: 0.7270 - val_acc: 0.7316
Epoch 20/20
5003/5003 [==============================] - 60s 12ms/sample - loss: 0.3200 - acc: 0.8811 - val_loss: 0.7252 - val_acc: 0.7395

1962/1962 [==============================] - 7s 4ms/sample - loss: 0.7473 - acc: 0.7125
Test accuracy: 0.71253824
Test loss: 0.747275456770966

#I liked it better at lower, and then add more epochs, increase dropout because overfitting is a thing.

I forgot to download the graph outputs, but it was basically a straight lines after ~3 epochs along 0.72 accuracy and 0.86 loss
"""

"""
model_6

test_train_split = 0.75
batch_size = 64
epoch_number = 50
validation_split = 0.15
learning_rate = 1e-6
dropout_rate = 0.50

Epoch 49/50
5003/5003 [==============================] - 59s 12ms/sample - loss: 0.5778 - acc: 0.7256 - val_loss: 0.6859 - val_acc: 0.6840
Epoch 50/50
5003/5003 [==============================] - 60s 12ms/sample - loss: 0.5838 - acc: 0.7234 - val_loss: 0.6876 - val_acc: 0.6795


1962/1962 [==============================] - 7s 4ms/sample - loss: 0.7073 - acc: 0.6213
Test accuracy: 0.6213048
Test loss: 0.7072517683503582
"""

"""
model_7 

test_train_split = 0.75
batch_size = 64
epoch_number = 50
validation_split = 0.15
learning_rate = 3e-6
dropout_rate = 0.50

implemented tensorboard during this guy, so can go there to see the runs data and graphs. I think on this next one I want to
build a different architecture. There was that paper Gautham posted from 2018 and it woould be neat to use their working arch.
But since it is lunch time, I will make a fwe small changed and hit go. I am mainly testing some more tensorboard features on
this model. but also, fuck it, I added another dense layer just for shits and giggles.

I'm pretty sure since I can get my training accuracy well over 90% (perhaps by overtraining?) that this is unneccessary
because this high accuracy means that I am fully able to map the training space. But also-- what the fuck do I know lol?

I also increased batchsize apparently.

also, auroc isnt working, something abput py.func being deprecated in tf2.0; look that guy up the error said it was a simple
fix
"""

In [ ]:
#all above are models using a different architecture. all below are comments from the current architecture

In [ ]:
#source list
"""
https://fizzylogic.nl/2017/05/08/monitor-progress-of-your-keras-based-neural-network-using-tensorboard/

https://stackoverflow.com/questions/41032551/how-to-compute-receiving-operating-characteristic-roc-and-auc-in-keras

https://astrobites.org/2018/07/16/creating-a-more-general-deep-learning-algorithm-for-galaxies/
"""